In [13]:
import json
import os
import pandas as pd
import numpy as np
import scipy.stats
from scipy import stats, ndimage
from scipy.signal import butter, lfilter, medfilt

#Change these for the data

trialLength = 3300
baselineLength = 1000

In [14]:
def calculate_column_wise_average(df):
    columns_to_average = df.columns[5:3305].values
    return df[columns_to_average].mean(axis=0)

In [17]:
all_participants_data = pd.DataFrame()
all_ext_participants_data = pd.DataFrame()
path = "/Users/dhewitt/Data/pps/"

def process_participant(participantID):
    global all_participants_data 
    global all_ext_participants_data
    
    PKL_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_dataframelist.pkl")
    CSV_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_Extractednew.csv")
    OUT_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_GazeObjProcessed.csv")
    AV_OUT_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_GazeObjProcessed_noAV.csv")
    EXT_AV_OUT_FILE_NAME = os.path.join(path + "P" + participantID + "/P" + participantID + "_GazeObjProcessed_AVext.csv")

    data2load = pd.read_csv(CSV_FILE_NAME) 
    dataframe_list = pd.read_pickle(PKL_FILE_NAME)

    #Interpolating the df due to changes in sampling rate for collecting data between trials.
    #Normalising the timestamp from the first timepoint.
    #Just getting the 'in trial' bit from trial start/cue appearing to the pain stimulus onset.

    # Create a mapping for converting 'gaze_object' strings to numeric values
    gaze_object_mapping = {'Terrain': 0, 'CONDITION_OBJECT(Clone)': 1}

    GazeObject = list(map(lambda df: np.interp(np.arange(baselineLength+trialLength), df['timestamp'] - df['timestamp'][0], 
                   df['gaze_object'].map(gaze_object_mapping).to_numpy()), dataframe_list))
    
    GazeObject=pd.DataFrame(GazeObject)
    
    GazeObject = GazeObject.drop(GazeObject.iloc[:, 0:3800],axis = 1)
    columns = dict(map(reversed, enumerate(GazeObject.columns))) #resetting the index
    GazeObject = GazeObject.rename(columns=columns)
    
    # Now getting the condition data

    df = pd.DataFrame(data2load)
    newdf = pd.concat([df,GazeObject],axis=1)

    # Creating averages for each column (i.e., over all trials for each condition)

    condMiddle = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & (newdf['condorder'] == 1)]
    Middle_cond_average1 = calculate_column_wise_average(condMiddle)

    condLeft = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & (newdf['condorder'] == 1)]
    Left_cond_average1 = calculate_column_wise_average(condLeft)

    condRight = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & (newdf['condorder'] == 1)]
    Right_cond_average1 = calculate_column_wise_average(condRight) 

    extMiddle = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & (newdf['condorder'] == 2)]
    Middle_ext_average1 = calculate_column_wise_average(extMiddle)

    extLeft = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & (newdf['condorder'] == 2)]
    Left_ext_average1 = calculate_column_wise_average(extLeft)

    extRight = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & (newdf['condorder'] == 2)]
    Right_ext_average1 = calculate_column_wise_average(extRight)    
    
    #### second run
    condMiddle = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & (newdf['condorder'] == 3)]
    Middle_cond_average2 = calculate_column_wise_average(condMiddle)

    condLeft = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & (newdf['condorder'] == 3)]
    Left_cond_average2 = calculate_column_wise_average(condLeft)

    condRight = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & (newdf['condorder'] == 3)]
    Right_cond_average2 = calculate_column_wise_average(condRight) 

    extMiddle = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & (newdf['condorder'] == 4)]
    Middle_ext_average2 = calculate_column_wise_average(extMiddle)

    extLeft = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & (newdf['condorder'] == 4)]
    Left_ext_average2 = calculate_column_wise_average(extLeft)

    extRight = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & (newdf['condorder'] == 4)]
    Right_ext_average2 = calculate_column_wise_average(extRight)   

    # Saving single subject data for pain-cue-block comparisons

    frames = [Middle_cond_average1, Left_cond_average1, Right_cond_average1, Middle_ext_average1, Left_ext_average1, Right_ext_average1,
             Middle_cond_average2, Left_cond_average2, Right_cond_average2, Middle_ext_average2, Left_ext_average2, Right_ext_average2]
    avscore = pd.concat(frames, axis=1, keys = ["CM1", "CL1", "CR1", "EM1", "EL1", "ER1",
                                               "CM2", "CL2", "CR2", "EM2", "EL2", "ER2"])
    avscore.to_csv(AV_OUT_FILE_NAME)
    
    # Saving single subject data for extinction-tonic pain comparisons
    
    extMiddle1 = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & (newdf['condorder'] == 2)]
    Middle_ext_average1 = calculate_column_wise_average(extMiddle1)

    extLeft1 = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & (newdf['condorder'] == 2)]
    Left_ext_average1 = calculate_column_wise_average(extLeft1)

    extRight1 = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & (newdf['condorder'] == 2)]
    Right_ext_average1 = calculate_column_wise_average(extRight1) 
    
    extMiddle2 = newdf.loc[(newdf['start_new_trial_condition'] == 'MiddleLow') & (newdf['condorder'] == 4)]
    Middle_ext_average2 = calculate_column_wise_average(extMiddle2)

    extLeft2 = newdf.loc[(newdf['start_new_trial_condition'] == 'Left') & (newdf['condorder'] == 4)]
    Left_ext_average2 = calculate_column_wise_average(extLeft2)

    extRight2 = newdf.loc[(newdf['start_new_trial_condition'] == 'Right') & (newdf['condorder'] == 4)]
    Right_ext_average2 = calculate_column_wise_average(extRight2)
    
    frames = [Middle_ext_average1, Left_ext_average1, Right_ext_average1, Middle_ext_average2, Left_ext_average2, Right_ext_average2]
    avscore = pd.concat(frames, axis=1, keys = ["EM1", "EL1", "ER1", "EM2", "EL2", "ER2"])
    avscore.to_csv(EXT_AV_OUT_FILE_NAME)
    
    # Read the CSV file for the current participant
    participant_data = pd.read_csv(AV_OUT_FILE_NAME)
    participant_data["Participant"] = participantID
    all_participants_data = pd.concat([all_participants_data, participant_data], ignore_index=True)
    
    ext_participant_data = pd.read_csv(EXT_AV_OUT_FILE_NAME)
    ext_participant_data["Participant"] = participantID
    all_ext_participants_data = pd.concat([all_ext_participants_data, ext_participant_data], ignore_index=True)    

In [19]:
# List of participant IDs
participant_ids = ["02", "03", "04", "05", "06", "08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "19", "20", "21", "22", "24", "25", "26", "27", "28", "29","30"] 



# Loop over each participant
for participantID in participant_ids:
    process_participant(participantID)
    
# Concatenate all participant data into a single DataFrame
all_participants_data.to_csv("/Users/dhewitt/Data/pps/Exports/Combined_GazeObjectData_noav.csv", index=False)
all_ext_participants_data.to_csv("/Users/dhewitt/Data/pps/Exports/Combined_GazeObjectExt.csv", index=False)

KeyboardInterrupt: 